# nlp_q1.ipynb BagOfWordsModel

Review Input:
BagOfWordsModel 
{u'text': u"Love it!!!!! Love it!!!!!! love it!!!!!!!   Who doesn't love Culver's!"}
BagOfWordsModel 
{u'text': u'Everything was great except for the burgers they are greasy and very charred compared to other stores.'}
BagOfWordsModel 
{u'text': u'I really like both Chinese restaurants in town.  This one has outstanding crab rangoon.  Love the chicken with snow peas and mushrooms and General Tso Chicken.  Food is always ready in 10 minutes which is accurate.  Good place and they give you free pop.'}
BagOfWordsModel 
{u'text': u'Above average takeout with friendly staff. The sauce on the pan fried noodle is tasty. Dumplings are quite good.'}
BagOfWordsModel 
{u'text': u"We order from Chang Jiang often and have never been disappointed.  The menu is huge, and can accomodate anyone's taste buds.  The service is quick, usually ready in 10 minutes."}


In [4]:
import simplejson as json
import random
from sklearn.cross_validation import train_test_split
import numpy as np
import pandas as pd
import nltk
pd.options.display.max_rows = 999
from sklearn.externals import joblib


In [84]:
## Constants

PATH_PKL = '/home/vagrant/miniprojects/questions/3_week/nlp/pickles/'

In [85]:

lst_reviews = []
lst_stars = []
lst_len = []

with open("/home/vagrant/miniprojects/questions/3_week/nlp/data/yelp_train_academic_dataset_review.json") as f:
    for line in f:
        tmp = json.loads(line)
        lst_reviews.append(tmp['text'])
        lst_stars.append(tmp['stars'])
        lst_len.append(len(tmp['text']))
# reviews come out tokenized- each review is 1 string, but not
# vectorized
print len(lst_stars), "reviews"


1012913 reviews


In [50]:
## Define stopwords ###

# import nltk
# stops = nltk.corpus.stopwords.words('english')

# nltk corpus may not be available on heroku:
stops = [u'i', u'me', u'my', u'myself', u'we', u'our', u'ours', u'ourselves', u'you', u'your', u'yours', u'yourself', u'yourselves', u'he', u'him', u'his', u'himself', u'she', u'her', u'hers', u'herself', u'it', u'its', u'itself', u'they', u'them', u'their', u'theirs', u'themselves', u'what', u'which', u'who', u'whom', u'this', u'that', u'these', u'those', u'am', u'is', u'are', u'was', u'were', u'be', u'been', u'being', u'have', u'has', u'had', u'having', u'do', u'does', u'did', u'doing', u'a', u'an', u'the', u'and', u'but', u'if', u'or', u'because', u'as', u'until', u'while', u'of', u'at', u'by', u'for', u'with', u'about', u'against', u'between', u'into', u'through', u'during', u'before', u'after', u'above', u'below', u'to', u'from', u'up', u'down', u'in', u'out', u'on', u'off', u'over', u'under', u'again', u'further', u'then', u'once', u'here', u'there', u'when', u'where', u'why', u'how', u'all', u'any', u'both', u'each', u'few', u'more', u'most', u'other', u'some', u'such', u'no', u'nor', u'not', u'only', u'own', u'same', u'so', u'than', u'too', u'very', u's', u't', u'can', u'will', u'just', u'don', u'should', u'now']

In [86]:
# bag of words 
import nltk
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stops, min_df=100, max_df=1.0, max_features=100000)

counts = cv.fit_transform(lst_reviews)
X = counts
y = lst_stars
print counts.shape

(1012913, 18164)


In [87]:
## Delete stop words to reduce file size
delattr(cv, 'stop_words_')

In [88]:
joblib.dump(bag_of_words_vectorizer, PATH_PKL + 'q1_cv.pkl')

['/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q1_cv.pkl']

In [56]:
## Save massive vectorized array & vectorizer
joblib.dump(counts, PATH_PKL + 'q1_arr_counts.pkl')

['/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q1_cv.pkl']

In [61]:
## Cross Validate ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

print X_train.shape, len(y_train)
print X_test.shape, len(y_test)


(759684, 18164) 759684
(253229, 18164) 253229


In [70]:
# optimize a regresser (LinearRegression)
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV

parameters = {'alpha':[.01, .1, .2, .5, 1.]}
rgr = GridSearchCV(Ridge(), parameters)

X = X_train[0:100000,:]
y = y_train[0:100000]

rgr.fit(X,y)

GridSearchCV(cv=None,
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'alpha': [0.01, 0.1, 0.2, 0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None, scoring=None,
       verbose=0)

In [74]:
print rgr.best_params_
print rgr.best_estimator_
print rgr.best_score_
print rgr.get

{'alpha': 1.0}
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001)
0.282398408295
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001)


In [78]:
# Train the optimized regressor
ridge = Ridge(alpha=1.0)

ridge.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, solver='auto', tol=0.001)

In [83]:
# save the regresser model
joblib.dump(ridge,'/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q1_lrr.pkl')

['/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q1_lrr.pkl',
 '/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q1_lrr.pkl_01.npy']

In [8]:
ridge = joblib.load('/home/vagrant/miniprojects/questions/3_week/nlp/pickles/q1_lrr.pkl')

In [82]:
p = ridge.predict(X_test) # define a transform method in clf to take raw strings
print p

[ 4.12257203  2.90971301  3.68648693 ...,  4.44168538  3.73465754
  3.96391873]


In [80]:
ridge.score(X_test, y_test)

0.50892793653387214

In [ ]:
lst_q1 = [
            "Love it!!!!! Love it!!!!!! love it!!!!!!!   Who doesn't love Culver's!",
    'Everything was great except for the burgers they are greasy and very charred compared to other stores.',
    'I really like both Chinese restaurants in town.  This one has outstanding crab rangoon.  Love the chicken with snow peas and mushrooms and General Tso Chicken.  Food is always ready in 10 minutes which is accurate.  Good place and they give you free pop.',
    'Above average takeout with friendly staff. The sauce on the pan fried noodle is tasty. Dumplings are quite good.',
    "We order from Chang Jiang often and have never been disappointed.  The menu is huge, and can accomodate anyone's taste buds.  The service is quick, usually ready in 10 minutes."
]

In [32]:
## From Validation Input

t1 = bag_of_words_vectorizer.transform(["Love it!!!!! Love it!!!!!! love it!!!!!!!   Who doesn't love Culver's!"])
print clf.predict(t1)

t1 = bag_of_words_vectorizer.transform(['Everything was great except for the burgers they are greasy and very charred compared to other stores.'])
print clf.predict(t1)

t1 = bag_of_words_vectorizer.transform(['I really like both Chinese restaurants in town.  This one has outstanding crab rangoon.  Love the chicken with snow peas and mushrooms and General Tso Chicken.  Food is always ready in 10 minutes which is accurate.  Good place and they give you free pop.'])
print clf.predict(t1)

t1 = bag_of_words_vectorizer.transform(['Above average takeout with friendly staff. The sauce on the pan fried noodle is tasty. Dumplings are quite good.'])
print clf.predict(t1)

t1 = bag_of_words_vectorizer.transform(["We order from Chang Jiang often and have never been disappointed.  The menu is huge, and can accomodate anyone's taste buds.  The service is quick, usually ready in 10 minutes."])
print clf.predict(t1)

[5]
[4]
[4]
[3]
[5]
